In [55]:
train_num = 2200
test_num = 5



In [59]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
embedder = SentenceTransformer('bert-base-chinese')

Exception when trying to download https://sbert.net/models/bert-base-chinese.zip. Response 404


In [60]:
df = pd.read_csv('.//data//Result_TFIDF.csv', index_col=0)
df.head()

,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
0,臺灣桃園地方法院,2018-04-18T00:00:00+08:00,"106,易,1093",刑事,妨害公務,判決,邱建豪犯妨害公務執行罪，累犯，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣桃園地方法院刑事判決106年度易字第1093號公訴人臺灣桃園地方法院檢察署檢察官被告邱建...,邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃...
1,臺灣臺中地方法院,2011-05-19T00:00:00+08:00,"100,中交簡,1002",刑事,公共危險,判決,王炎權服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事簡易判決100年度中交簡字第1002號聲請人臺灣臺中地方法院檢察署檢察官...,王炎權於民國100年4月8日晚間9時30分許起至翌日凌晨3時許止，在彰化縣芬園鄉○○路○段2...
2,臺灣臺中地方法院,2011-01-25T00:00:00+08:00,"100,中交簡,100",刑事,公共危險,判決,林耀明服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾伍日，如易科罰金，以新臺幣壹仟元折...,臺灣臺中地方法院刑事簡易判決100年度中交簡字第100號聲請人臺灣臺中地方法院檢察署檢察官被...,林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市○○區○○路與豐樂路交岔路口附...
3,臺灣臺中地方法院,2011-05-12T00:00:00+08:00,"100,中交簡,1016",刑事,公共危險,判決,朱建鴻犯不能安全駕駛動力交通工具罪，處罰金新台幣伍萬元，如易服勞役，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事簡易判決100年度中交簡字第1016號聲請人臺灣臺中地方法院檢察署檢察官...,朱建鴻於民國100年4月19日晚間9時許起，在臺中市大雅區清泉崗友人住處內飲用啤酒，至同日晚...
4,臺灣臺中地方法院,2011-06-13T00:00:00+08:00,"100,中交簡,1106",刑事,公共危險,判決,洪國棋服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事簡易判決100年度中交簡字第1106號聲請人臺灣臺中地方法院檢察署檢察官...,洪國棋於民國100年4月28日晚間7時許起至同日晚間8時30分許止，在臺中市○里區○○路21...


In [61]:
df1 = df.sample(n=train_num)
test = df.sample(n=test_num)

In [62]:
#embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Corpus with example sentences
#corpus = ['A man is eating food.',
#          'A man is eating a piece of bread.',
#          'The girl is carrying a baby.',
#          'A man is riding a horse.',
#          'A woman is playing violin.',
#          'Two men pushed carts through the woods.',
#          'A man is riding a white horse on an enclosed ground.',
#          'A monkey is playing drums.',
#          'A cheetah is running behind its prey.'
#          ]
corpus = []
for index, row in df1.iterrows():
    if (row.Situation == None):
        continue
    if (isinstance(row.Situation, float)):
        continue
    if (len(row.Situation) == 0):
        continue
    corpus.append(row.Situation)
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
print(len(corpus))

In [63]:
# Query sentences:
#queries = ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']
queries = []
for index, row in test.iterrows():
    queries.append(str(row.Situation))
print(len(queries))

5


In [64]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    print(query)
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))

温惠瑛於民國105年9月4日下午3時30分許，在臺中市烏日區之友人住處，飲用啤酒之酒類後，基於飲酒後駕駛動力交通工具之犯意，猶騎車牌號碼000-000號普通重型機車上路。嗣於同日下午6時41分許，行經臺中市○○區○○○○巷0號前時，因行車搖擺不定，為執行巡邏勤務之員警予以攔檢盤查後，發現其身上有酒味，經警於同日下午6時59分許，對其施以酒精濃度之吐氣測試，當場測得其吐氣所含酒精濃度達每公升0.62毫克，而查悉上情。




Query: 温惠瑛於民國105年9月4日下午3時30分許，在臺中市烏日區之友人住處，飲用啤酒之酒類後，基於飲酒後駕駛動力交通工具之犯意，猶騎車牌號碼000-000號普通重型機車上路。嗣於同日下午6時41分許，行經臺中市○○區○○○○巷0號前時，因行車搖擺不定，為執行巡邏勤務之員警予以攔檢盤查後，發現其身上有酒味，經警於同日下午6時59分許，對其施以酒精濃度之吐氣測試，當場測得其吐氣所含酒精濃度達每公升0.62毫克，而查悉上情。

Top 5 most similar sentences in corpus:
温惠瑛於民國105年9月4日下午3時30分許，在臺中市烏日區之友人住處，飲用啤酒之酒類後，基於飲酒後駕駛動力交通工具之犯意，猶騎車牌號碼000-000號普通重型機車上路。嗣於同日下午6時41分許，行經臺中市○○區○○○○巷0號前時，因行車搖擺不定，為執行巡邏勤務之員警予以攔檢盤查後，發現其身上有酒味，經警於同日下午6時59分許，對其施以酒精濃度之吐氣測試，當場測得其吐氣所含酒精濃度達每公升0.62毫克，而查悉上情。 (Score: 1.0000)
鄭妃萍於民國105年4月1日晚間11時50分許起至105年4月2日凌晨2時30分許止，在臺中市烏日區光華街某小吃部，飲用啤酒之酒類後，基於飲酒後駕駛動力交通工具之犯意，猶騎車牌號碼000-000號普通重型機車上路。嗣行經臺中市○○區○○路0段00號前時，因行車不穩，為執行巡邏勤務之員警予以攔檢盤查後，發現其身上有酒味，經警於105年4月2日凌晨3時25分許，對其施以酒精濃度之吐氣測試，當場測得其吐氣所含酒精濃度達每公升0.54毫克，而查悉上情。 (Score: 0.9908)
林國盛於民國104年12月12日晚間8時許起至同日晚間11時許止，在臺中市東區旱溪街某友人住處內，飲用啤酒之酒類後，基於飲